# Best Place for Children in Toronto!

I am Kevin Huang. The notebook is for the final project of the Coursera course - Applied Data Science Capstone. The topic is __Best Place for Children in Toronto!__

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Introduction

For newcomers to Toronto that have many children, the most important things is to make sure their sons/daughters have enough spaces to play. Living in a neighborhoods with many __parks__, __trails__, and __playgrounds__ nearby is optimal for such families. With the clustering approach, the goal of this project is to find the recommended neighborhoods for them.

## Data

1. Wikipedia page that lists the neighborhoods in Toronto.
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

2. Geospatial data of Toronto neighborhoods.
http://cocl.us/Geospatial_data

3. Foursquare location data to search for venues nearby each Toronto neighborhood.